In [26]:
from pprint import pprint
from convertmol import parse_sdf_file, bond_type_dict, single_bond_stereo_dict, double_bond_stereo_dict
from torch_geometric.data import Data
from torch_geometric.nn import *
from torch_geometric.utils import to_dense_adj, to_dense_batch, add_self_loops
from torch_geometric.nn.conv import MessagePassing
import torch
from torch import nn
import rdkit
from tqdm import tqdm
import itertools
from rdkit import Chem
import pandas as pd
from importlib import reload
import matplotlib.pyplot as plt
from rdkit import RDLogger
from copy import deepcopy
from torch.utils.data import Dataset, DataLoader
# Suppress RDKit warnings
RDLogger.DisableLog('rdApp.*')
import sascorer
torch.set_default_device('cpu:0')#'cuda'

In [29]:
from neo4j import GraphDatabase, RoutingControl


URI = "bolt://localhost:7687"
AUTH = ("neo4j", "password")


def add_friend(driver, name, friend_name):
    driver.execute_query(
        "MERGE (a:Person {name: $name}) "
        "MERGE (friend:Person {name: $friend_name}) "
        "MERGE (a)-[:KNOWS]->(friend)",
        name=name, friend_name=friend_name, database_="neo4j",
    )


def print_friends(driver, name):
    records, _, _ = driver.execute_query(
        "MATCH (a:Person)-[:KNOWS]->(friend) WHERE a.name = $name "
        "RETURN friend.name ORDER BY friend.name",
        name=name, database_="neo4j", routing_=RoutingControl.READ,
    )
    for record in records:
        print(record["friend.name"])



In [30]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    print("A")

A


In [31]:
import convertmol
import importlib
importlib.reload(convertmol)

<module 'convertmol' from 'C:\\Users\\umarzein\\Desktop\\biocheminformatics\\convertmol.py'>

In [43]:
import re
atom_re=re.compile("^atom\d+$")#.match("watom123")
bond_re=re.compile("^bond\d+$")

def create_mol(tx,d,source):
    assert isinstance(source, str), "source must be str"
    atoms={}
    bonds={}
    props={}
    for k,v in d.items():
        if atom_re.match(k):
            atoms[k]=v
        elif bond_re.match(k):
            bonds[k]=v
        else:
            props[k]=v
    tx.run(f"MERGE (:Source {{name: '{source}'}});")
    
    mol_props = ", ".join([f"{key}: ${key}" for key in props.keys()])
    query = f"CREATE (:Molecule {{{mol_props}}});"
    tx.run(query, props)
    for k, atom in atoms.items():
        atom_props = ", ".join([f"{key}: ${key}" for key in atom.keys()])
        query = f"MATCH (m: Molecule {{id: '{props['id']}'}}) CREATE (a:Atom {{id: '{k}', {atom_props}}})-[:PART_OF]->(m);"
        tx.run(query, atom)
    for k, bond in bonds.items():
        orig=bond.pop("orig")
        dest=bond.pop("dest")
        bond_props = ", ".join([f"{key}: ${key}" for key in bond.keys()])
        query = f"MATCH (orig:Atom {{id: '{orig}'}})-[:PART_OF]->(m: Molecule {{id: '{props['id']}'}}), \
            (dest:Atom {{id: '{dest}'}})-[:PART_OF]->(m: Molecule {{id: '{props['id']}'}}) \
            CREATE \
                (orig)-[:BOND {{id: '{k}', {bond_props}}}]->(dest);"
        #query = f"MATCH (orig:Atom {{id: '{orig}'}})-[:PART_OF]->(m: Molecule {{id: '{props['id']}'}}), (dest:Atom {{id: '{dest}'}}) \
        #    CREATE \
        #        (orig)-[:BOND {{id: '{k}', {bond_props}}}]->(dest), \
        #        (dest)-[:BOND {{id: '{k}', {bond_props}}}]->(orig);"
        tx.run(query, bond)
    
    tx.run(f"MATCH (m: Molecule {{id: '{props['id']}'}}),(s:Source {{name: '{source}'}}) CREATE (m)-[:PART_OF]->(s);")
    return


In [44]:

# Use the driver
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session() as session:
        for m in parse_sdf_file("./samples.sdf", data_items=True):
            print("processing")
            #display(m)
            session.execute_write(create_mol, m, "sample_gdb9")
            print("done")
            

processing
done
processing
done
processing
done
processing
done
processing
done


In [36]:

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session() as session:
        display(session.run("MATCH (n:Molecule)-[:PART_OF]->(s:Source) return n").data())

[{'n': {'software': '-OEChem-03231823243D',
   'id': 'gdb_1',
   'version': 'V2000'}},
 {'n': {'software': '-OEChem-03231823233D',
   'id': 'gdb_2',
   'version': 'V2000'}},
 {'n': {'software': '-OEChem-03231823243D',
   'id': 'gdb_3',
   'version': 'V2000'}},
 {'n': {'software': '-OEChem-03231823253D',
   'id': 'gdb_4',
   'version': 'V2000'}},
 {'n': {'software': 'ChemDraw2024',
   'LogP': '0.83',
   'id': 'CO2',
   'version': 'V2000'}}]

In [95]:

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session() as session:
        result=(session.run("MATCH (a:Atom)-[b:BOND]->(a2:Atom)-[:PART_OF]->(m:Molecule{id:'CO2'}) return a,b,a2"))
        for row in result:
            out=(row['b'])

In [96]:
dict(out)

{'bond_stereo': 'Use_coordinates', 'id': 'bond1', 'bond_type': 'Double'}

In [52]:
tmp[0]['b']

({'symbol': 'C',
  'charge': 'outside_limits',
  'stereo_parity': 'not_stereo',
  'mass_diff': '0',
  'x': -0.72,
  'y': 0.0,
  'valence': 'no marking',
  'z': 0.0,
  'id': 'atom1'},
 'BOND',
 {'symbol': 'O',
  'charge': 'outside_limits',
  'stereo_parity': 'not_stereo',
  'mass_diff': '0',
  'x': 0.72,
  'y': 0.0,
  'valence': 'no marking',
  'z': 0.0,
  'id': 'atom2'})

In [74]:
import threading
from time import sleep, time
sem = threading.Semaphore(2)
def fun1():
    n=3
    while n>0:
        sleep(0.00000001)
        with sem:
            n-=1
            print(1)
            sleep(1)

def fun2():
    n=3
    while n>0:
        sleep(0.00000001)
        with sem:
            n-=1
            print(2)
            sleep(1)
start=time()
t = threading.Thread(target = fun1)
t.start()
t2 = threading.Thread(target = fun2)
t2.start()
t.join()
t2.join()
print("took",time()-start,"seconds")

1
2
2
1
21

took 3.0773708820343018 seconds
